# Unsupervised Learning 2 

## Lesson 3: Market Basket Analysis

### Part 1: Identifying the 'Small' Customers

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load in the datasets identified in the previous lesson as small customers
df = pd.read_csv("small_cust.csv")

In [3]:
# Display some simple information to get a big picture of the data
df.head()

,CustomerID,Recency,Frequency,Monetary
0,12821,215,6,92.72
1,12823,75,5,1759.50
2,12829,337,11,293.00
3,12831,263,9,215.05
4,12833,146,24,417.38


In [4]:
df.describe()

,CustomerID,Recency,Frequency,Monetary
count,1318.000000,1318.000000,1318.000000,1318.000000
mean,15575.327011,163.324734,14.071320,283.795561
std,1612.855454,101.604452,10.491899,261.844992
min,12821.000000,2.000000,1.000000,3.750000
25%,14162.750000,68.250000,6.250000,141.520000
50%,15553.000000,160.000000,12.000000,221.890000
75%,16989.500000,250.000000,19.750000,343.577500
max,18282.000000,363.000000,77.000000,4055.720000


In [5]:
df.shape

(1318, 4)

In [6]:
# Get the total number of transactions for these customers
sum(df.Frequency)

18546

### Part 2: Getting the transactions

In [7]:
# Read in the filtered transaction data since the RFM was generated from this
tr = pd.read_csv("OnlineRetail_Filtered.csv")
print(tr.shape)

(339702, 8)


In [8]:
tr[tr.Description == 'POSTAGE'].shape

(27, 8)

In [9]:
# Filter dataframe to keep only transactions from the small customers
tf_f = tr[tr['CustomerID'].isin(df.CustomerID)]

In [10]:
tf_f.shape # Validate that transaction count is the same

(18546, 8)

In [11]:
# These 2 features will be used to group the data.
# Description is used instead of StockID as it is human readable when we analyze the assoc rules later
tf_f['Description'] = tf_f['Description'].str.strip()
tf_f['InvoiceNo'] = tf_f['InvoiceNo'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
# The result should be 1 row per transaction. Each column is the quantiy purchased for that item
basket = (tf_f.groupby(['InvoiceNo','Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

#drop postage entries
basket.drop('POSTAGE', inplace=True, axis=1)

In [13]:
'POSTAGE' in (basket.columns)

False

In [14]:
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,...,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
538368,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538470,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
538509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Part 3: Generating the Association Rules

In [17]:
# import libraries
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [18]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

# We only need binary encoding for the assoc rules
basket_sets = basket.applymap(encode_units)

In [19]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [20]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ASSORTED COLOUR BIRD ORNAMENT),(PAINTED METAL PEARS ASSORTED),0.050741,0.023945,0.019384,0.382022,15.953986,0.018169,1.579434
1,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.023945,0.050741,0.019384,0.809524,15.953986,0.018169,4.983609
2,(BAKING SET SPACEBOY DESIGN),(BAKING SET 9 PIECE RETROSPOT),0.019954,0.042189,0.015964,0.800000,18.962162,0.015122,4.789054
3,(BAKING SET 9 PIECE RETROSPOT),(BAKING SET SPACEBOY DESIGN),0.042189,0.019954,0.015964,0.378378,18.962162,0.015122,1.576595
4,(BLUE HAPPY BIRTHDAY BUNTING),(PINK HAPPY BIRTHDAY BUNTING),0.015393,0.014253,0.011403,0.740741,51.970370,0.011183,3.802166


In [21]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ].sort_values(['lift','confidence'], ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5,(PINK HAPPY BIRTHDAY BUNTING),(BLUE HAPPY BIRTHDAY BUNTING),0.014253,0.015393,0.011403,0.800000,51.970370,0.011183,4.923033
38,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.016534,0.031357,0.014823,0.896552,28.591850,0.014305,9.363550
8,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.024515,0.031357,0.019954,0.813953,25.957717,0.019186,5.206457
33,"(PINK REGENCY TEACUP AND SAUCER, REGENCY CAKES...",(GREEN REGENCY TEACUP AND SAUCER),0.014823,0.031357,0.011973,0.807692,25.758042,0.011508,5.036944
2,(BAKING SET SPACEBOY DESIGN),(BAKING SET 9 PIECE RETROSPOT),0.019954,0.042189,0.015964,0.800000,18.962162,0.015122,4.789054
1,(PAINTED METAL PEARS ASSORTED),(ASSORTED COLOUR BIRD ORNAMENT),0.023945,0.050741,0.019384,0.809524,15.953986,0.018169,4.983609
